# Construindo Agentes com Memória usando DSPy ReAct e Mem0

Este tutorial demonstra como construir agentes conversacionais inteligentes que podem lembrar informações através de múltiplas interações usando o framework ReAct do DSPy combinado com as capacidades de memória do [Mem0](https://docs.mem0.ai/).

## O Que Você Vai Construir

Ao final deste tutorial, você terá um agente habilitado com memória que pode:

- **Lembrar preferências do usuário** e conversas passadas
- **Armazenar e recuperar informações factuais** sobre usuários e tópicos
- **Usar memória para informar decisões** e fornecer respostas personalizadas
- **Lidar com conversas complexas multi-turno** com consciência de contexto
- **Gerenciar diferentes tipos de memórias** (fatos, preferências, experiências)

## Pré-requisitos

- Entendimento básico de DSPy e agentes ReAct
- Python 3.9+ instalado
- API keys para seu provedor LLM preferido

## Arquitetura do Sistema

O sistema que vamos construir consiste em três componentes principais:

1. **Mem0 Memory System**: Camada de armazenamento e recuperação de memórias
2. **Memory Tools**: Interface para interagir com o sistema de memória
3. **ReAct Agent**: Agente que usa ferramentas de memória para tomar decisões


## Instalação e Setup

Primeiro, vamos instalar as dependências necessárias:


In [ ]:
# Instalação das dependências
%pip install -q dspy mem0ai openai


## Passo 1: Configuração Inicial e Integração Mem0

Mem0 fornece uma camada de memória que pode armazenar, buscar e recuperar memórias para agentes de IA. Vamos começar entendendo como integrá-lo com DSPy:

### Como o Mem0 Funciona:

- **Armazenamento vetorial**: Converte memórias em embeddings para busca semântica
- **Isolamento por usuário**: Mantém memórias separadas para diferentes usuários
- **Busca contextual**: Recupera memórias relevantes baseadas em similaridade semântica
- **Persistência**: Memórias são mantidas entre sessões


In [ ]:
import dspy
from mem0 import Memory
import os
from typing import List, Dict, Any, Optional
from datetime import datetime
import time

# Configurar variável de ambiente (substitua pela sua API key)
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

# Configuração do sistema de memória Mem0
config = {
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4o-mini",
            "temperature": 0.1  # Baixa temperatura para respostas mais consistentes
        }
    },
    "embedder": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-small"  # Modelo eficiente para embeddings
        }
    }
}

print("Configuração Mem0 criada com sucesso!")
print(f"   - LLM: {config['llm']['config']['model']}")
print(f"   - Embedder: {config['embedder']['config']['model']}")


## Passo 2: Criar Ferramentas para Interação com Memória

Vamos criar um conjunto de ferramentas que permitem ao agente interagir com o sistema de memória. Cada ferramenta terá uma responsabilidade específica:

### Ferramentas Principais:

1. **`store_memory`**: Armazena nova informação na memória
2. **`search_memories`**: Busca memórias relevantes por similaridade semântica
3. **`get_all_memories`**: Recupera todas as memórias de um usuário
4. **`update_memory`**: Atualiza uma memória existente
5. **`delete_memory`**: Remove uma memória específica


In [ ]:
class MemoryTools:
    """Ferramentas para interagir com o sistema de memória Mem0."""

    def __init__(self, memory: Memory):
        """Inicializa as ferramentas de memória."""
        self.memory = memory

    def store_memory(self, content: str, user_id: str = "default_user") -> str:
        """Armazena informação na memória."""
        try:
            self.memory.add(content, user_id=user_id)
            return f"Stored memory: {content}"
        except Exception as e:
            return f"Error storing memory: {str(e)}"

    def search_memories(self, query: str, user_id: str = "default_user", limit: int = 5) -> str:
        """Busca memórias relevantes usando similaridade semântica."""
        try:
            results = self.memory.search(query, user_id=user_id, limit=limit)
            if not results or "results" not in results or not results["results"]:
                return "No relevant memories found."

            memory_text = "Relevant memories found:\\n"
            for i, result in enumerate(results["results"], 1):
                memory_text += f"{i}. {result['memory']}\\n"
            return memory_text
        except Exception as e:
            return f"Error searching memories: {str(e)}"

    def get_all_memories(self, user_id: str = "default_user") -> str:
        """Recupera todas as memórias de um usuário."""
        try:
            results = self.memory.get_all(user_id=user_id)
            if not results or "results" not in results or not results["results"]:
                return "No memories found for this user."

            memory_text = "All memories for user:\\n"
            for i, result in enumerate(results["results"], 1):
                memory_text += f"{i}. {result['memory']}\\n"
            return memory_text
        except Exception as e:
            return f"Error retrieving memories: {str(e)}"

    def update_memory(self, memory_id: str, new_content: str) -> str:
        """Atualiza uma memória existente."""
        try:
            self.memory.update(memory_id, new_content)
            return f"Updated memory with new content: {new_content}"
        except Exception as e:
            return f"Error updating memory: {str(e)}"

    def delete_memory(self, memory_id: str) -> str:
        """Deleta uma memória específica."""
        try:
            self.memory.delete(memory_id)
            return "Memory deleted successfully."
        except Exception as e:
            return f"Error deleting memory: {str(e)}"


def get_current_time() -> str:
    """Obtém a data e hora atual."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


print("Ferramentas de memória criadas com sucesso!")
print("   Funções disponíveis:")
print("   - store_memory: Armazena nova informação")
print("   - search_memories: Busca memórias relevantes")
print("   - get_all_memories: Recupera todas as memórias")
print("   - update_memory: Atualiza memória existente")
print("   - delete_memory: Remove memória específica")
print("   - get_current_time: Obtém data/hora atual")


## Passo 3: Construir o Agente ReAct com Memória

Agora vamos criar nosso agente principal que combina o framework ReAct do DSPy com as ferramentas de memória.

### Componentes do Agente:

1. **Signature**: Define a interface do agente (entrada/saída)
2. **Tools**: Conjunto de ferramentas que o agente pode usar
3. **ReAct Module**: Motor de raciocínio que decide quando usar cada ferramenta

### Como o ReAct Funciona:

O ReAct (Reasoning + Acting) alterna entre:
- **Raciocínio**: Pensar sobre o problema e decidir próxima ação
- **Ação**: Executar uma ferramenta
- **Observação**: Analisar o resultado da ação

Este loop continua até o agente ter informação suficiente para responder.


In [ ]:
class MemoryQA(dspy.Signature):
    """Signature para agente conversacional com memória que armazena e recupera informações."""
    user_input: str = dspy.InputField(desc="Entrada do usuário")
    response: str = dspy.OutputField(desc="Resposta do assistente")


class MemoryReActAgent(dspy.Module):
    """Agente ReAct aprimorado com capacidades de memória Mem0."""

    def __init__(self, memory: Memory):
        """Inicializa o agente com sistema de memória."""
        super().__init__()
        self.memory_tools = MemoryTools(memory)

        # Criar lista de ferramentas para ReAct
        self.tools = [
            self.memory_tools.store_memory,
            self.memory_tools.search_memories,
            self.memory_tools.get_all_memories,
            get_current_time,
            self.set_reminder,
            self.get_preferences,
            self.update_preferences,
        ]

        # Inicializar ReAct com nossas ferramentas
        self.react = dspy.ReAct(
            signature=MemoryQA,
            tools=self.tools,
            max_iters=6  # Máximo de iterações de raciocínio-ação
        )

    def forward(self, user_input: str):
        """Processa entrada do usuário com raciocínio baseado em memória."""
        return self.react(user_input=user_input)

    def set_reminder(self, reminder_text: str, date_time: str = None, user_id: str = "default_user") -> str:
        """Define um lembrete para o usuário."""
        if date_time:
            reminder = f"Reminder set for {date_time}: {reminder_text}"
        else:
            reminder = f"Reminder: {reminder_text}"
        return self.memory_tools.store_memory(
            f"REMINDER: {reminder}", 
            user_id=user_id
        )

    def get_preferences(self, category: str = "general", user_id: str = "default_user") -> str:
        """Obtém preferências do usuário para uma categoria específica."""
        query = f"user preferences {category}"
        return self.memory_tools.search_memories(
            query=query,
            user_id=user_id
        )

    def update_preferences(self, category: str, preference: str, user_id: str = "default_user") -> str:
        """Atualiza preferências do usuário."""
        preference_text = f"User preference for {category}: {preference}"
        return self.memory_tools.store_memory(
            preference_text,
            user_id=user_id
        )


print("Agente ReAct com memória criado com sucesso!")
print("   Capacidades:")
print("   - Armazenar e recuperar memórias")
print("   - Gerenciar preferências do usuário")
print("   - Criar e gerenciar lembretes")
print("   - Manter contexto entre conversas")


## Passo 4: Executar e Testar o Agente

Vamos criar uma demonstração completa do agente em ação. Testaremos:

1. **Armazenamento de preferências**: O agente aprende sobre gostos do usuário
2. **Recuperação de memória**: O agente lembra informações passadas
3. **Criação de lembretes**: O agente pode agendar lembretes
4. **Conversação contextual**: O agente usa memória para personalizar respostas

### Fluxo da Demonstração:

Simularemos uma conversa com Alice onde:
- Ela compartilha suas preferências alimentares
- Ela menciona seus hábitos de exercício
- Ela pede para o agente lembrar informações
- Ela solicita lembretes
- O agente demonstra que lembra de tudo compartilhado


In [ ]:
def run_memory_agent_demo():
    """Demonstração do agente ReAct com memória."""

    # Configurar DSPy
    lm = dspy.LM(model='openai/gpt-4o-mini')
    dspy.configure(lm=lm)

    # Inicializar sistema de memória
    memory = Memory.from_config(config)

    # Criar nosso agente
    agent = MemoryReActAgent(memory)

    # Conversas de exemplo demonstrando capacidades de memória
    print("\\n" + "="*60)
    print("Demonstração: Agente ReAct com Memória")
    print("="*60)
    print("\\nNesta demonstração, Alice irá:")
    print("1. Compartilhar suas preferências alimentares")
    print("2. Informar seus hábitos de exercício")
    print("3. Testar a memória do agente")
    print("4. Criar lembretes")
    print("5. Verificar tudo que o agente lembra")
    print("\\n" + "="*60 + "\\n")

    conversations = [
        "Hi, I'm Alice and I love Italian food, especially pasta carbonara.",
        "I'm Alice. I prefer to exercise in the morning around 7 AM.",
        "I'm Alice. What do you remember about my food preferences?",
        "I'm Alice. Set a reminder for me to go grocery shopping tomorrow.",
        "I'm Alice. What are my exercise preferences?",
        "I'm Alice. I also enjoy hiking on weekends.",
        "I'm Alice. What do you know about me so far?"
    ]

    for i, user_input in enumerate(conversations, 1):
        print(f"\\n{'─'*60}")
        print(f"Interação {i}/7")
        print(f"{'─'*60}")
        print(f"\\nAlice: {user_input}")
        print("\\nProcessando...")

        try:
            response = agent(user_input=user_input)
            print(f"\\nAgente: {response.response}")
            time.sleep(1)  # Pausa para melhor visualização

        except Exception as e:
            print(f"\\nErro: {e}")
            import traceback
            traceback.print_exc()

    print("\\n" + "="*60)
    print("Demonstração concluída!")
    print("="*60)
    print("\\nResumo:")
    print("   - O agente armazenou múltiplas informações sobre Alice")
    print("   - Recuperou memórias quando solicitado")
    print("   - Criou lembretes baseados em contexto")
    print("   - Demonstrou consciência contextual completa")


# Executar a demonstração
print("Iniciando demonstração do agente com memória...\\n")
run_memory_agent_demo()


## Resultado Esperado

Quando você executar a célula acima, deverá ver uma saída similar a:

```
Demonstração: Agente ReAct com Memória
============================================================

Interação 1/7
────────────────────────────────────────────────────────────

Alice: Hi, I'm Alice and I love Italian food, especially pasta carbonara.

Agente: Hi Alice! It's great to meet you! Pasta carbonara is a delicious 
choice. Do you have a favorite place to get it, or do you enjoy making it 
at home?

...

Interação 7/7
────────────────────────────────────────────────────────────

Alice: I'm Alice. What do you know about me so far?

Agente: So far, I know that you enjoy hiking on weekends, prefer to 
exercise in the morning around 7 AM, and love Italian food, especially pasta 
carbonara. Additionally, you have a reminder set for grocery shopping.
```

### O Que Observar:

1. **Primeira interação**: O agente armazena informações sobre Alice automaticamente
2. **Interações subsequentes**: O agente recupera e usa memórias anteriores
3. **Última interação**: O agente demonstra lembrança completa de tudo compartilhado

### Capacidades Demonstradas:

- **Armazenamento automático**: O agente identifica e armazena informações relevantes
- **Recuperação contextual**: Busca memórias relevantes para cada pergunta
- **Personalização**: Usa memórias para personalizar respostas
- **Continuidade**: Mantém contexto através de múltiplas interações


## Explorando o Sistema de Memória

Vamos fazer alguns experimentos adicionais para entender melhor como o sistema de memória funciona:


In [ ]:
# Configurar DSPy e Mem0 para experimentação
lm = dspy.LM(model='openai/gpt-4o-mini')
dspy.configure(lm=lm)
memory = Memory.from_config(config)
memory_tools = MemoryTools(memory)

print("Sistema de memória inicializado para experimentação\\n")

# Experimento 1: Armazenar diferentes tipos de memória
print("Experimento 1: Armazenando diferentes tipos de informação\\n")
print("─" * 60)

test_memories = [
    "Bob loves playing tennis every Saturday",
    "Bob is allergic to peanuts",
    "Bob works as a software engineer at TechCorp",
    "Bob's favorite programming language is Python",
]

for mem in test_memories:
    result = memory_tools.store_memory(mem, user_id="bob")
    print(f"OK: {result}")

print("\\n" + "─" * 60)
print("Memórias armazenadas com sucesso!\\n")


In [ ]:
# Experimento 2: Buscar memórias por similaridade semântica
print("Experimento 2: Busca semântica de memórias\\n")
print("─" * 60)

search_queries = [
    "What sports does Bob like?",
    "Tell me about Bob's job",
    "Does Bob have any dietary restrictions?",
]

for query in search_queries:
    print(f"\\nQuery: {query}")
    results = memory_tools.search_memories(query, user_id="bob", limit=2)
    print(f"\\n{results}")
    print("─" * 60)

print("\\nBusca semântica concluída!")


In [ ]:
# Experimento 3: Visualizar todas as memórias
print("Experimento 3: Visualizando todas as memórias do usuário\\n")
print("─" * 60)

all_memories = memory_tools.get_all_memories(user_id="bob")
print(all_memories)

print("─" * 60)
print("\\nTodas as memórias recuperadas!")


## Análise e Trade-offs

### Vantagens do Sistema de Memória

1. **Personalização**: Respostas adaptadas ao contexto e histórico do usuário
2. **Continuidade**: Conversas coerentes através de múltiplas sessões
3. **Proatividade**: Capacidade de sugerir ações baseadas em padrões
4. **Escalabilidade**: Isolamento por usuário permite sistemas multi-tenant

### Limitações e Considerações

1. **Latência**: Busca em memória adiciona overhead a cada interação
2. **Custo**: Embeddings e armazenamento têm custos associados
3. **Privacidade**: Dados sensíveis requerem cuidado especial
4. **Qualidade**: Memórias incorretas podem propagar erros

### Quando Usar Memória

**Use quando:**
- Aplicações com usuários recorrentes
- Necessidade de personalização
- Conversas multi-turno complexas
- Assistentes pessoais ou corporativos

**Evite quando:**
- Interações únicas e desconexas
- Latência é crítica
- Orçamento limitado
- Dados extremamente sensíveis sem infraestrutura apropriada

### Melhores Práticas

1. **Implementar expiração de memórias**: Dados antigos podem ficar irrelevantes
2. **Categorizar memórias**: Facilita busca e organização
3. **Validar informações**: Confirmar fatos importantes com usuários
4. **Monitorar custos**: Embeddings e storage podem escalar rapidamente
5. **GDPR/Privacy**: Implementar direito ao esquecimento e controle de dados


## Próximos Passos

Para transformar este sistema em uma solução production-ready, considere:

### 1. Persistência de Dados

```python
# Integrar com PostgreSQL ou MongoDB
config = {
    "vector_store": {
        "provider": "pgvector",
        "config": {
            "host": "localhost",
            "database": "memories"
        }
    }
}
```

### 2. Gestão de Memória

- Implementar políticas de expiração
- Criar sistema de tags/categorias
- Adicionar versionamento de memórias
- Implementar compressão para memórias antigas

### 3. Segurança e Privacidade

- Criptografia de memórias sensíveis
- Auditoria de acesso
- Compliance com GDPR/LGPD
- Anonimização de dados quando apropriado

### 4. Otimização de Performance

- Cache de memórias frequentemente acessadas
- Busca assíncrona
- Batch processing para múltiplas queries
- Índices otimizados para busca vetorial

### 5. Observabilidade

- Métricas de uso de memória
- Rastreamento de queries
- Alertas para anomalias
- Dashboard de analytics


## Recursos Adicionais

### Documentação

- [Mem0 Documentation](https://docs.mem0.ai/)
- [DSPy ReAct Framework](https://dspy-docs.vercel.app/)
- [Vector Databases Comparison](https://www.pinecone.io/learn/vector-database/)

### Papers Relevantes

- Yao, S., et al. (2022). **ReAct: Synergizing Reasoning and Acting in Language Models**. arXiv:2210.03629
- Shinn, N., et al. (2023). **Reflexion: Language Agents with Verbal Reinforcement Learning**. arXiv:2303.11366

### Projetos Open Source

- [LangChain Memory](https://python.langchain.com/docs/modules/memory/)
- [Zep Memory Store](https://github.com/getzep/zep)
- [MemGPT](https://github.com/cpacker/MemGPT)


## Conclusão

Neste tutorial, exploramos como construir agentes inteligentes com memória usando DSPy e Mem0. Aprendemos:

1. **Integração Mem0**: Como configurar e usar o sistema de memória
2. **Ferramentas de Memória**: Criar interfaces para operações CRUD
3. **ReAct + Memória**: Combinar raciocínio com capacidades de memória
4. **Casos de Uso**: Aplicações práticas como assistentes pessoais
5. **Trade-offs**: Análise de vantagens, limitações e quando usar

### Pontos-Chave

- **Memória é essencial** para agentes conversacionais realistas
- **DSPy + Mem0** fornece uma stack poderosa e flexível
- **Busca semântica** permite recuperação inteligente de contexto
- **Production-ready** requer considerações adicionais de segurança e escala

### Próxima Jornada

Agora que você domina os fundamentos, explore:
- Multi-agent systems com memória compartilhada
- Otimização de prompts com feedback de memória
- Integração com ferramentas externas (APIs, databases)
- Fine-tuning de modelos usando dados de memória

---

**Construa agentes que realmente lembram e aprendem!**
